In [1]:
import os 

os.chdir(os.path.abspath(os.path.pardir))

In [2]:
from app.clients.frame_finder import FrameFinder
from app.metrics.iou import IOUCalculator
from app.metrics.map import MAPCalculator
from app.metrics.recognition_metrics import VideoIdentificationEvaluator

In [ ]:
# Init frame finder

json_files=[]
folder_path = "results/aaivoninskaya avzotova_4 anaalesuslova"

for filename in os.listdir(folder_path):
    full_path = os.path.join(folder_path, filename)

    if os.path.isfile(full_path) and filename.lower().endswith(".json"):
        json_files.append(full_path)
frame_finder = FrameFinder(json_files=json_files)

In [ ]:
iou_calculator = IOUCalculator("annotations.xml", frame_finder)

stats = iou_calculator.get_summary_stats()

print("Person detection metrics:")
print(f"IoU mean: {stats['person_iou']['mean']:.3f} ± {stats['person_iou']['std']:.3f}")
print(f"Count difference: {stats['person_count_diff']['mean']:.1f}")

print("\nFace detection metrics:")
print(f"IoU mean: {stats['face_iou']['mean']:.3f} ± {stats['face_iou']['std']:.3f}")
print(f"Count difference: {stats['face_count_diff']['mean']:.1f}")

Person detection metrics:
IoU mean: 0.764 ± 0.078
Count difference: 0.1

Face detection metrics:
IoU mean: 0.633 ± 0.109
Count difference: -0.5


In [ ]:
map_calculator = MAPCalculator("annotations.xml", frame_finder, iou_threshold=0.5)

metrics = map_calculator.calculate_map()

print("Metrics:")
for class_name, value in metrics.items():
    print(f"{class_name}: {value:.4f}")

Metrics:
AP_face_bbox: 0.7712
AP_person_bbox: 0.9936
mAP: 0.8824


In [7]:
def generate_ground_truth(frame_patterns, total_frames: int):
    pattern_frames = sorted(frame_patterns.keys())
    patterns = [frame_patterns[frame] for frame in pattern_frames]
    
    ground_truth = {}
    pattern_index = 0
    
    for frame in range(total_frames):
        if frame in frame_patterns:
            current_pattern = frame_patterns[frame]
        else:
            current_pattern = patterns[pattern_index % len(patterns)]
            pattern_index += 1
        
        ground_truth[frame] = current_pattern
    
    return ground_truth

In [ ]:
frame_patterns = {
    # 0: ["dkorolev", "mvmoiseev", "amzayakina"],
    # 0: ["ogmenkov"],
    0: ["aaivoninskaya", "avzotova_4", "anaalesuslova"],
    # 0: ["saslastnikov1"],

    
    
    
    # 15: ["ikmityushkin", "dkorolev", "mvmoiseev", "amzayakina"],
    # 30: [""dkorolev", "mvmoiseev""]
}

# total_frames = 2880
total_frames = 3600


ground_truth = generate_ground_truth(frame_patterns, total_frames)

In [ ]:
evaluator = VideoIdentificationEvaluator(
    known_identities=os.listdir("datasets/backup"),
    frame_finder=frame_finder
)

results = evaluator.process_video(ground_truth)

evaluator.print_summary(["aaivoninskaya", "avzotova_4", "anaalesuslova"])